In [3]:
import scipy.io.wavfile as wav
import numpy as np

# Function to convert stereo audio to mono
def stereo_to_mono(audio_data):
    return audio_data.mean(axis=1)

# Function to apply a PEQ filter
def apply_peq_filter(signal, fs, fc, fbw, g):
    T = 1 / fs
    wc = 2 * np.pi * fc
    wbw = 2 * np.pi * fbw
    wcd = (2 / T) * np.tan(wc * (T / 2))
    Q = wcd / wbw

    # Input coefficients
    a0 = 4 + 2 * (g / Q) * wcd * T + (wcd * T) ** 2
    a1 = 2 * (wcd * T) ** 2 - 8
    a2 = 4 - 2 * (g / Q) * wcd * T + (wcd * T) ** 2

    # Output coefficients
    b0 = 4 + 2 * (1 / Q) * wcd * T + (wcd * T) ** 2
    b1 = -(2 * (wcd * T) ** 2 - 8)
    b2 = -(4 - 2 * (1 / Q) * wcd * T + (wcd * T) ** 2)

    filtered_signal = np.zeros(len(signal))
    for s in range(2, len(signal)):
        filtered_signal[s] = (1 / b0) * ((a0 * signal[s] + a1 * signal[s - 1] + a2 * signal[s - 2]) + (b1 * filtered_signal[s - 1] + b2 * filtered_signal[s - 2]))

    return filtered_signal

# Read WAV file
fs, data = wav.read(r'C:\Users\kalin\Desktop\IIR_Filter_Basics\1.wav')

# Convert to mono if stereo
if data.ndim > 1:
    data = stereo_to_mono(data)

# Apply PEQ1 filter (Example settings)
fc1 = 3500  # Hz
fbw1 = 500  # Hz
g1 = 100     # Gain
peq1_filtered = apply_peq_filter(data, fs, fc1, fbw1, g1)

# Apply PEQ2 filter (Example settings)
fc2 = 100   # Hz
fbw2 = 100   # Hz
g2 = 0.1   # Gain
peq2_filtered = apply_peq_filter(data, fs, fc2, fbw2, g2)

# Write the processed audio back to new WAV files
wav.write(r'C:\Users\kalin\Desktop\IIR_Filter_Basics\output1.wav', fs, np.int16(peq1_filtered))
wav.write(r'C:\Users\kalin\Desktop\IIR_Filter_Basics\output2.wav', fs, np.int16(peq2_filtered))
